# budget_tables_draft2017b.ipynb

## Purpose
Produce tables of MARC aerosol budget for *"Effective radiative forcing in the aerosol-climate model CAM5.3-MARC-ARG compared to default CAM5.3"*, based on results from project [p17c-marc-comparison](https://github.com/grandey/p17c-marc-comparison).

## Requirements
- NetCDF files produced by [budget_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/budget_cdo_nco_draft2017b.ipynb) and [analysis_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/analysis_cdo_nco_draft2017b.ipynb).
- Python modules mentioned in cell #2, inlcuding [climapy](https://github.com/grandey/climapy) ([doi:10.5281/zenodo.1053020](https://doi.org/10.5281/zenodo.1053020)).

## Author
Benjamin S. Grandey, 2018

## Acknowledgments
Thanks are due to **Alex Avramov**.  The interpretation of the diagnostics are informed by *print_MARC_budget_Benjamin.pro*, an IDL script written by Alex Avramov.

In [1]:
! date

Tue Jun 19 16:15:35 +08 2018


In [2]:
import climapy
import os
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import xarray as xr

# Print versions of packages
for module in [climapy, np, pd, scipy, xr]:
    try:
        print('{}.__version__ = {}'.format(module.__name__, module.__version__))
    except AttributeError:
        pass

# Default precision when displaying DataFrames
pd.set_option('precision', 3)

climapy.__version__ = 0.1.0+b35937c
numpy.__version__ = 1.14.3
pandas.__version__ = 0.23.0
scipy.__version__ = 1.1.0
xarray.__version__ = 0.10.4


## Post-processed data file locations
Location of files written by [budget_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/budget_cdo_nco_draft2017b.ipynb) and [analysis_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/analysis_cdo_nco_draft2017b.ipynb). The latter contains the column burdens.

In [3]:
budget_dir = os.path.expandvars('$HOME/data/projects/p17c_marc_comparison/budget_cdo_nco_draft2017b/')
analysis_dir = os.path.expandvars('$HOME/data/projects/p17c_marc_comparison/analysis_cdo_nco_draft2017b/')

## Read global totals into dictionary

In [4]:
# # Diagnostic variables to load
# variable_list = ['mOCSF', 'mBCSF',  # surface emissions
#                  'DST01SF', 'DST02SF', 'DST03SF', 'DST04SF',
#                  'SSLT01SF', 'SSLT02SF', 'SSLT03SF', 'SSLT04SF',
#                  'ctOC_SF', 'ctBC_SF', 'ctDST_SF', 'ctSSLT_SF',
#                  'mACCDRY', 'mAITDRY', 'mNUCDRY',  # dry deposition
#                  'mOCDRY', 'mMOSDRY', 'mOIMDRY', 'mBCDRY', 'mMBSDRY', 'mBIMDRY',
#                  'DST01DD', 'DST02DD', 'DST03DD', 'DST04DD',
#                  'SSLT01DD', 'SSLT02DD', 'SSLT03DD', 'SSLT04DD',
#                  'ctSUL_DRY', 'ctOC_DRY', 'ctBC_DRY', 'ctDST_DRY', 'ctSSLT_DRY']
# for variable in [
#         'mACCWET', 'mAITWET', 'mNUCWET',  # wet deposition
#         'mOCWET', 'mMOSWET', 'mOIMWET', 'mBCWET', 'mMBSWET', 'mBIMWET',
#         'DST01PP', 'DST02PP', 'DST03PP', 'DST04PP', 'SSLT01PP', 'SSLT02PP', 'SSLT03PP', 'SSLT04PP',
#         'tSUL_WET', 'tOC_WET', 'tBC_WET', 'tDST_WET', 'tSSLT_WET',
#         'AGEmOC', 'AGEmMOS', 'AGEmBC', 'AGEmMBS', # aging
#         'tSUL_AGE',
#         'mACCACT', 'mAITACT', 'mNUCACT',  # nucleation scavenging by stratiform clouds
#         'mOCACT', 'mMOSACT', 'mOIMACT', 'mBCACT', 'mMBSACT', 'mBIMACT',
#         'tSUL_ACT', 'tOC_ACT', 'tBC_ACT',
#         'mACCSCV', 'mAITSCV', 'mNUCSCV',  # nucleation scavenging by convection
#         'mOCSCV', 'mMOSSCV', 'mOIMSCV', 'mBCSCV', 'mMBSSCV', 'mBIMSCV',
#         'tSUL_SCV', 'tOC_SCV', 'tBC_SCV',
#         'mSO4evap',  # SO4 cloud evaporation -> ACC
#         'tSUL_EVAP',
#         'BNUCmNUC',  # binary nucleation -> NUC
#         'tSUL_NUC',
#         'CNDmACC', 'CNDmAIT', 'CNDmNUC', 'CNDmMOS', 'CNDmMBS',  # H2SO4 condensation
#         'tSUL_CND',
#         'MVmNUC', 'MVmAIT',  # mass mode adjustment, NUC->AIT, AIT->ACC
#         'CGmNUCAC', 'CGmNUCAI', 'CGmNUCMB', 'CGmNUCMO',  # coagulation
#         'CGmAITAC', 'CGmAITMB', 'CGmAITMO', 'CGmACCMB', 'CGmACCMO', 'CGmOCMOS']:
#     variable_list.append('c{}'.format(variable))
# for aerosol in ['OC', 'MOS', 'OIM', 'BC', 'MBS', 'BIM', 'NUC', 'AIT', 'ACC',
#                 'DST01', 'DST02', 'DST03', 'DST04', 'SSLT01', 'SSLT02', 'SSLT03', 'SSLT04',
#                 'tSUL', 'tOC', 'tBC', 'tDST', 'tSSLT']:
#     variable_list.append('c{}_LDG'.format(aerosol))  # loading
# for aerosol in ['tSUL', 'tOC', 'tBC', 'tDST', 'tSSLT']:
#     variable_list.append('c{}_SOURCE'.format(aerosol))  # sum of sources
#     variable_list.append('c{}_SINK'.format(aerosol))  # sum of sinks
# print('variable_list = {}'.format(variable_list))

In [5]:
# Diagnostics to load data for
diagnostic_dict = {'c{}_LDG': 'Burden, Tg',  # loading / burden
                   # Sources
                   'c{}_SOURCE': 'Sources, Tg/yr',  # sum of sources
                   'c{}_SF': 'Emissions, Tg/yr',
                   'c{}_CND': 'H2SO4 condensation, Tg/yr',
                   'c{}_BNUC': 'Binary nucleation, Tg/yr',
                   'c{}_EVAP': 'Cloud evaporation, Tg/yr',
                   'c{}_CGsrc': 'Coagulation (source), Tg/yr',
                   'c{}_AGEsrc': 'Aging (source), Tg/yr',
                   'c{}_ADJsrc': 'Mode mass adjustment (source), Tg/yr',
                   # Sinks
                   'c{}_SINK': 'Sinks, Tg/yr',  # sum of sinks
                   'c{}_DRY': 'Dry deposition, Tg/yr',
                   'c{}_IMP': 'Impaction scavenging, Tg/yr',
                   'c{}_ACT': 'Nucleation scavenging by stratiform clouds, Tg/yr',
                   'c{}_SCV': 'Nucleation scavenging by convective clouds, Tg/yr',
                   'c{}_CGsnk': 'Coagulation (sink), Tg/yr',
                   'c{}_AGEsnk': 'Aging (sink), Tg/yr',
                   'c{}_ADJsnk': 'Mode mass adjustment (sink), Tg/yr',
                   }

In [6]:
# List of variables (aerosol-diagnostic combinations) to try loading data for
variable_list = []
for aerosol in ['tSUL', 'tOC', 'tBC', 'tDST', 'tSSLT',
                'NUC', 'AIT', 'ACC', 'OC', 'MOS', 'OIM', 'SIMOS', 'BC', 'MBS', 'BIM', 'SIMBS']:
    for diagnostic in diagnostic_dict.keys():
        variable_list.append(diagnostic.format(aerosol)) 

In [7]:
# Read data and calculate global totals, standard errors
# Initialise dictionaries
mean_dict = {}  # mean global totals
error_dict = {}  # standard errors, based on annual means
# Loop over years and variables
for year in ['2000', '1850']:
    for variable in variable_list:
        variable_year = (variable, year)
        try:
            # Read data from input file
            if '_LDG' in variable:
                in_filename = '{}/marc_{}_{}_ANN.nc'.format(analysis_dir, year, variable)
                in_ds = xr.open_dataset(in_filename, decode_times=False)[variable]
            else:
                in_filename = '{}/marc_{}_{}_ANN.nc'.format(budget_dir, year, variable)
                in_ds = xr.open_dataset(in_filename, decode_times=False)[variable]
            # Exclude first two years as spin-up
            data = in_ds.isel(time=slice(2, None))
            # Calculate sum across earth: kg/m2/s -> kg/s / kg/m2 -> kg
            totals = climapy.xr_area_weighted_stat(data, stat='sum')
            # Convert units
            if '_LDG' in variable:  # kg -> Tg
                totals = totals / 1e9
            else:  # kg/s -> Tg/yr
                totals = totals / 1e9 * 60 * 60 *24 * 365
            # Calculate mean across years
            mean_dict[variable_year] = float(totals.mean())
            # Calculate standard error
            error = totals.std(dim='time', ddof=1).data.flatten() / np.sqrt(totals['time'].size)
            error_dict[variable_year] = float(error)
            # Print mean
            print('{:13}, {}: {:10.4f} ± {:7.4f}'.format(variable, year, mean_dict[variable_year],
                                                        error_dict[variable_year]))
            # Close input file
            in_ds.close()   
        except FileNotFoundError:
            pass
! date

ctSUL_LDG    , 2000:     1.3229 ±  0.0028
ctSUL_SOURCE , 2000:   536.2118 ±  0.9161
ctSUL_CND    , 2000:    11.9424 ±  0.0525
ctSUL_BNUC   , 2000:     0.0021 ±  0.0000
ctSUL_EVAP   , 2000:   502.3945 ±  0.9012
ctSUL_AGEsrc , 2000:    21.8728 ±  0.0296
ctSUL_SINK   , 2000:  -536.3833 ±  0.9209
ctSUL_DRY    , 2000:    -5.2176 ±  0.0135
ctSUL_IMP    , 2000:  -116.3788 ±  0.0696
ctSUL_ACT    , 2000:  -394.1190 ±  0.8484
ctSUL_SCV    , 2000:   -20.6679 ±  0.0484
ctOC_LDG     , 2000:     1.3289 ±  0.0019
ctOC_SOURCE  , 2000:   100.7154 ±  0.0000
ctOC_SF      , 2000:   100.7154 ±  0.0000
ctOC_SINK    , 2000:  -100.6576 ±  0.0079
ctOC_DRY     , 2000:    -4.0990 ±  0.0061
ctOC_IMP     , 2000:   -83.3608 ±  0.0243
ctOC_ACT     , 2000:   -12.7048 ±  0.0214
ctOC_SCV     , 2000:    -0.4930 ±  0.0023
ctBC_LDG     , 2000:     0.0924 ±  0.0001
ctBC_SOURCE  , 2000:     6.8859 ±  0.0000
ctBC_SF      , 2000:     6.8859 ±  0.0000
ctBC_SINK    , 2000:    -6.8921 ±  0.0009
ctBC_DRY     , 2000:    -0.3703 ± 

cAIT_CGsrc   , 1850:     0.0000 ±  0.0000
cAIT_ADJsrc  , 1850:     0.0062 ±  0.0000
cAIT_SINK    , 1850:    -0.0784 ±  0.0006
cAIT_DRY     , 1850:    -0.0000 ±  0.0000
cAIT_IMP     , 1850:    -0.0003 ±  0.0000
cAIT_ACT     , 1850:    -0.0005 ±  0.0000
cAIT_SCV     , 1850:    -0.0000 ±  0.0000
cAIT_CGsnk   , 1850:    -0.0001 ±  0.0000
cAIT_ADJsnk  , 1850:    -0.0774 ±  0.0006
cACC_LDG     , 1850:     0.4042 ±  0.0008
cACC_SOURCE  , 1850:   204.4547 ±  0.2586
cACC_CND     , 1850:     3.3633 ±  0.0112
cACC_EVAP    , 1850:   201.0139 ±  0.2570
cACC_CGsrc   , 1850:     0.0001 ±  0.0000
cACC_ADJsrc  , 1850:     0.0774 ±  0.0006
cACC_SINK    , 1850:  -204.5214 ±  0.2584
cACC_DRY     , 1850:    -1.8975 ±  0.0040
cACC_IMP     , 1850:   -46.1059 ±  0.0208
cACC_ACT     , 1850:  -145.9111 ±  0.2241
cACC_SCV     , 1850:   -10.3301 ±  0.0285
cACC_CGsnk   , 1850:    -0.2768 ±  0.0019
cOC_LDG      , 1850:     1.0681 ±  0.0015
cOC_SOURCE   , 1850:    80.8228 ±  0.0000
cOC_SF       , 1850:    80.8228 ± 

Note: the values are ~0.1% lower than Alex's values, due to using slightly different values for the area of the earth.

## Rearrange into DataFrames

In [8]:
# Initialise dataframes
columns = ['tSUL', 'tOC', 'tBC', 'tDST', 'tSSLT',
           'NUC', 'AIT', 'ACC', 'OC', 'MOS', 'OIM', 'SIMOS', 'BC', 'MBS', 'BIM', 'SIMBS']
mean_2000_df = pd.DataFrame(columns=columns)  # marc_2000
error_2000_df = pd.DataFrame(columns=columns)
mean_1850_df = pd.DataFrame(columns=columns)  # marc_1850
error_1850_df = pd.DataFrame(columns=columns)

In [9]:
# Populate with data for diagnostics
diagnostic_dict = {'c{}_LDG': 'Burden, Tg',
                   'c{}_SOURCE': 'Sources, Tg/yr',
                   'c{}_SF': 'Emissions, Tg/yr',
                   'c{}_CND': 'H2SO4 condensation, Tg/yr',
                   'c{}_BNUC': 'Binary nucleation, Tg/yr',
                   'c{}_EVAP': 'Cloud evaporation, Tg/yr',
                   'c{}_CGsrc': 'Coagulation (source), Tg/yr',
                   'c{}_AGEsrc': 'Aging (source), Tg/yr',
                   'c{}_ADJsrc': 'Mode mass adjustment (source), Tg/yr', 
                   'c{}_SINK': 'Sinks, Tg/yr',
                   'c{}_DRY': 'Dry deposition, Tg/yr',
                   'c{}_IMP': 'Impaction scavenging, Tg/yr',
                   'c{}_ACT': 'Nucleation scavenging by stratiform clouds, Tg/yr',
                   'c{}_SCV': 'Nucleation scavenging by convective clouds, Tg/yr',
                   'c{}_CGsnk': 'Coagulation (sink), Tg/yr',
                   'c{}_AGEsnk': 'Aging (sink), Tg/yr',
                   'c{}_ADJsnk': 'Mode mass adjustment (sink), Tg/yr',
                   }
# Loop over diagnostics
for k, v in diagnostic_dict.items():
    # Loop over columns
    for col in columns:
        if (k.format(col), '2000') in mean_dict:
            mean_2000_df.loc[v, col] = mean_dict[(k.format(col), '2000')]
            error_2000_df.loc[v, col] = error_dict[(k.format(col), '2000')]
            mean_1850_df.loc[v, col] = mean_dict[(k.format(col), '1850')]
            error_1850_df.loc[v, col] = error_dict[(k.format(col), '1850')]

In [10]:
# Lifetime
mean_2000_df.loc['Lifetime, days'] = (mean_2000_df.loc['Burden, Tg'] /
                                      mean_2000_df.loc['Sinks, Tg/yr'].abs() * 365)
error_2000_df.loc['Lifetime, days'] = (mean_2000_df.loc['Lifetime, days'] *
                                       ((error_2000_df.loc['Burden, Tg'] /
                                         mean_2000_df.loc['Burden, Tg'])**2 +
                                        (error_2000_df.loc['Sinks, Tg/yr'] /
                                         mean_2000_df.loc['Sinks, Tg/yr'].abs())**2).pow(0.5))
mean_1850_df.loc['Lifetime, days'] = (mean_1850_df.loc['Burden, Tg'] /
                                      mean_1850_df.loc['Sinks, Tg/yr'] * -365)
error_1850_df.loc['Lifetime, days'] = (mean_1850_df.loc['Lifetime, days'] *
                                       ((error_1850_df.loc['Burden, Tg'] /
                                         mean_1850_df.loc['Burden, Tg'])**2 +
                                        (error_1850_df.loc['Sinks, Tg/yr'] /
                                         mean_1850_df.loc['Sinks, Tg/yr'].abs())**2).pow(0.5))

In [11]:
# Print means for marc_2000
mean_2000_df

,tSUL,tOC,tBC,tDST,tSSLT,NUC,AIT,ACC,OC,MOS,OIM,SIMOS,BC,MBS,BIM,SIMBS
"Burden, Tg",1.32,1.33,0.0924,40.7,9.61,3.37e-06,2.7e-05,0.94,1.05,0.593,0.28,0.313,0.0745,0.0877,0.0179,0.0698
"Sources, Tg/yr",536,101,6.89,3.65e+03,5.48e+03,0.01,0.103,507,101,49.1,23,26.1,6.89,8.21,1.79,6.42
"Emissions, Tg/yr",NaN,101,6.89,3.65e+03,5.48e+03,NaN,NaN,NaN,101,NaN,NaN,NaN,6.89,NaN,NaN,NaN
"H2SO4 condensation, Tg/yr",11.9,NaN,NaN,NaN,NaN,0.00799,0.0948,4.61,NaN,3.93,NaN,3.93,NaN,3.29,NaN,3.29
"Binary nucleation, Tg/yr",0.00206,NaN,NaN,NaN,NaN,0.00206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Cloud evaporation, Tg/yr",502,NaN,NaN,NaN,NaN,NaN,NaN,502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Coagulation (source), Tg/yr",NaN,NaN,NaN,NaN,NaN,NaN,1.76e-05,0.000133,NaN,3.84,1.88,1.96,NaN,1.41,NaN,1.41
"Aging (source), Tg/yr",21.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.3,21.2,20.2,NaN,3.5,1.79,1.71
"Mode mass adjustment (source), Tg/yr",NaN,NaN,NaN,NaN,NaN,NaN,0.00783,0.101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Sinks, Tg/yr",-536,-101,-6.89,-3.67e+03,-5.53e+03,-0.0101,-0.103,-507,-101,-49.2,-23.1,-26.1,-6.89,-8.23,-1.8,-6.43


In [12]:
# Print means for marc_1850
mean_1850_df

,tSUL,tOC,tBC,tDST,tSSLT,NUC,AIT,ACC,OC,MOS,OIM,SIMOS,BC,MBS,BIM,SIMBS
"Burden, Tg",0.519,1.17,0.0362,41.6,9.69,3.4e-06,2.09e-05,0.404,1.07,0.21,0.104,0.107,0.0346,0.00958,0.0016,0.00798
"Sources, Tg/yr",211,80.8,2.33,3.76e+03,5.51e+03,0.00699,0.0784,204,80.8,12.1,5.91,6.19,2.33,0.635,0.137,0.498
"Emissions, Tg/yr",NaN,80.8,2.33,3.76e+03,5.51e+03,NaN,NaN,NaN,80.8,NaN,NaN,NaN,2.33,NaN,NaN,NaN
"H2SO4 condensation, Tg/yr",4.72,NaN,NaN,NaN,NaN,0.00617,0.0722,3.36,NaN,0.985,NaN,0.985,NaN,0.292,NaN,0.292
"Binary nucleation, Tg/yr",0.00082,NaN,NaN,NaN,NaN,0.00082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Cloud evaporation, Tg/yr",201,NaN,NaN,NaN,NaN,NaN,NaN,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Coagulation (source), Tg/yr",NaN,NaN,NaN,NaN,NaN,NaN,1.49e-05,0.000104,NaN,0.867,0.666,0.202,NaN,0.0753,NaN,0.0753
"Aging (source), Tg/yr",5.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.3,5.25,5,NaN,0.268,0.137,0.131
"Mode mass adjustment (source), Tg/yr",NaN,NaN,NaN,NaN,NaN,NaN,0.00622,0.0774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Sinks, Tg/yr",-211,-80.8,-2.33,-3.78e+03,-5.56e+03,-0.00699,-0.0784,-205,-80.7,-12.2,-5.94,-6.21,-2.33,-0.637,-0.138,-0.5


In [13]:
! date

Tue Jun 19 16:15:43 +08 2018
